## Capítulo 02: Integração DuckDB + Arrow
**Curso:** Apache Arrow + DuckDB

### Tópicos abordados:
- Modos de integração
- Conversões bidirecionais
- Query em Arrow Tables
- Schemas e metadados
- Error handling

In [2]:
# -*- coding: utf-8 -*-
"""
Configuração inicial e imports
Nota: UTF-8 é configurado automaticamente em notebooks Jupyter
"""

import pyarrow as pa
import duckdb
import pandas as pd
import numpy as np

# Instalar dependências
print("Dependências necessárias:")
print("pip install pyarrow duckdb pandas numpy")

print("="*60)
print(f"CAPÍTULO 02: INTEGRAÇÃO DUCKDB + ARROW")
print("="*60)

Dependências necessárias:
pip install pyarrow duckdb pandas numpy
CAPÍTULO 02: INTEGRAÇÃO DUCKDB + ARROW


### Preparação dos Dados

In [ ]:
# Dados de exemplo globais
try:
    print("\nGerando dados de exemplo...")
    data = pa.table({
        'id': range(1000),
        'valor': np.random.randn(1000),
        'categoria': np.random.choice(['A', 'B', 'C'], 1000)
    })
    print(f"Tabela PyArrow criada: {data.num_rows} linhas")
except Exception as e:
    print(f"Erro ao criar dados: {e}")

# Conexão DuckDB
con = duckdb.connect()

### Tópico 1: Modos de integração

In [ ]:
print(f"\n--- {'Modos de integração'.upper()} ---")

# Modo 1: Query direta em Arrow table
result1 = con.execute("SELECT COUNT(*) as total FROM data").fetchone()
print(f"\n1. Query direta na Arrow table:")
print(f"   Total de registros: {result1[0]}")

# Modo 2: Usando duckdb.query() com Arrow table
result2 = duckdb.query("SELECT categoria, COUNT(*) as qtd FROM data GROUP BY categoria ORDER BY qtd DESC")
print(f"\n2. Usando duckdb.query():")
print(result2.to_df())

# Modo 3: Registrando Arrow table com nome customizado
con.register('my_arrow_data', data)
result3 = con.execute("SELECT AVG(valor) as media FROM my_arrow_data").fetchone()
print(f"\n3. Arrow table registrada:")
print(f"   Média dos valores: {result3[0]:.4f}")
con.unregister('my_arrow_data')

### Tópico 2: Conversões bidirecionais

In [ ]:
print(f"\n--- {'Conversões bidirecionais'.upper()} ---")

# 1. Arrow para DuckDB e retorna como Arrow
arrow_result = con.execute("SELECT * FROM data WHERE valor > 0 LIMIT 5").fetch_arrow_table()
print(f"\n1. DuckDB query retornando Arrow:")
print(f"   Tipo: {type(arrow_result)}")
print(f"   Linhas: {arrow_result.num_rows}")
print(f"   Colunas: {arrow_result.column_names}")

# 2. DuckDB para Pandas e depois para Arrow
pandas_result = con.execute("SELECT categoria, COUNT(*) as count FROM data GROUP BY categoria").df()
arrow_from_pandas = pa.Table.from_pandas(pandas_result)
print(f"\n2. DuckDB -> Pandas -> Arrow:")
print(arrow_from_pandas)

# 3. Round-trip: Arrow -> DuckDB -> Arrow
original_count = data.num_rows
processed = con.execute("SELECT * FROM data WHERE categoria = 'A'").fetch_arrow_table()
print(f"\n3. Round-trip conversion:")
print(f"   Original: {original_count} linhas")
print(f"   Filtrado: {processed.num_rows} linhas")
print(f"   Preservou tipos: {data.schema.equals(processed.schema)}")

### Tópico 3: Query em Arrow Tables

In [ ]:
print(f"\n--- {'Query em Arrow Tables'.upper()} ---")

# 1. Query SQL simples
result_simple = duckdb.query("""
    SELECT 
        categoria,
        COUNT(*) as quantidade,
        AVG(valor) as media_valor
    FROM data
    GROUP BY categoria
    ORDER BY quantidade DESC
""").fetch_arrow_table()
print(f"\n1. Agregação por categoria:")
print(result_simple.to_pandas())

# 2. Query com filtros complexos
result_complex = duckdb.query("""
    SELECT 
        categoria,
        MIN(valor) as min_valor,
        MAX(valor) as max_valor,
        STDDEV(valor) as desvio_padrao
    FROM data
    WHERE valor BETWEEN -1 AND 1
    GROUP BY categoria
""").fetch_arrow_table()
print(f"\n2. Estatísticas com filtro:")
print(result_complex.to_pandas())

# 3. Query com window functions
result_window = duckdb.query("""
    SELECT 
        categoria,
        valor,
        ROW_NUMBER() OVER (PARTITION BY categoria ORDER BY valor DESC) as rank
    FROM data
    QUALIFY rank <= 3
""").fetch_arrow_table()
print(f"\n3. Top 3 valores por categoria:")
print(f"   Total de resultados: {result_window.num_rows}")

### Tópico 4: Schemas e metadados

In [ ]:
print(f"\n--- {'Schemas e metadados'.upper()} ---")

# 1. Inspecionar schema da Arrow table
print(f"\n1. Schema da Arrow table:")
print(f"   Colunas: {data.schema.names}")
for field in data.schema:
    print(f"   - {field.name}: {field.type}")

# 2. Criar tabela com schema customizado
custom_schema = pa.schema([
    pa.field('user_id', pa.int32(), nullable=False),
    pa.field('username', pa.string()),
    pa.field('score', pa.float64()),
    pa.field('active', pa.bool_())
])

custom_data = pa.table({
    'user_id': [1, 2, 3],
    'username': ['alice', 'bob', 'carol'],
    'score': [95.5, 87.3, 92.1],
    'active': [True, True, False]
}, schema=custom_schema)

print(f"\n2. Tabela com schema customizado:")
print(custom_data.schema)

# 3. Verificar compatibilidade de schemas
result_schema = con.execute("SELECT * FROM custom_data").fetch_arrow_table().schema
print(f"\n3. Schema após query DuckDB:")
print(f"   Campos iguais: {custom_schema.names == result_schema.names}")

# 4. Metadados customizados
schema_with_metadata = custom_schema.with_metadata({
    'dataset': 'user_scores',
    'version': '1.0',
    'created': '2024-01-01'
})
print(f"\n4. Schema com metadados:")
print(f"   Metadados: {schema_with_metadata.metadata}")

### Tópico 5: Error handling

In [ ]:
print(f"\n--- {'Error handling'.upper()} ---")

# 1. Erro em query SQL inválida
print(f"\n1. Tratamento de query inválida:")
try:
    result = con.execute("SELECT coluna_inexistente FROM data").fetch_arrow_table()
except Exception as e:
    print(f"   Erro capturado: {type(e).__name__}")
    print(f"   Mensagem: {str(e)[:80]}...")

# 2. Erro em conversão de tipos incompatíveis
print(f"\n2. Conversão de tipos incompatíveis:")
try:
    bad_data = pa.table({
        'id': ['a', 'b', 'c'],  # strings ao invés de números
        'value': [1, 2, 3]
    })
    result = con.execute("SELECT id * 10 FROM bad_data").fetch_arrow_table()
except Exception as e:
    print(f"   Erro capturado: {type(e).__name__}")
    print(f"   Mensagem: {str(e)[:80]}...")

# 3. Erro em acesso a tabela não registrada
print(f"\n3. Acesso a tabela não registrada:")
try:
    result = con.execute("SELECT * FROM tabela_inexistente").fetch_arrow_table()
except Exception as e:
    print(f"   Erro capturado: {type(e).__name__}")
    print(f"   Mensagem: {str(e)[:80]}...")

# 4. Best practice: validação antes da query
print(f"\n4. Validação preventiva:")
def safe_query(connection, table_name, query):
    try:
        # Verificar se a tabela existe no contexto
        result = connection.execute(query).fetch_arrow_table()
        return result, None
    except Exception as e:
        return None, str(e)

result, error = safe_query(con, 'data', "SELECT COUNT(*) as total FROM data")
if error:
    print(f"   Erro: {error}")
else:
    print(f"   Sucesso! Total: {result.to_pydict()['total'][0]}")

### Conclusão

In [ ]:
print("\n" + "="*60)
print(f"Fim do Capítulo 02")
print("="*60)